In [3]:
import numpy as np
from scipy import stats
import math

SAMPLE SIZE CALULATOR

In [4]:
# Current rates (example)
p1 = 53/640   # 0.082812 (8.82%)
p2 = 45/1530   # 0.029412 (2.94%)

# For statistical power calculation, you need:
# 1. Baseline conversion rate (estimate from control)
# 2. Minimum effect size you want to detect
# 3. Statistical power (usually 80%)
# 4. Significance level (usually 5%)

def sample_size_calculator(p1, p2, alpha=0.05, power=0.8):
    """Calculate required sample size per group"""
    p_pooled = (p1 + p2) / 2
    effect_size = abs(p1 - p2)
    
    z_alpha = stats.norm.ppf(1 - alpha/2)  # 1.96 for 95% confidence
    z_beta = stats.norm.ppf(power)         # 0.84 for 80% power
    
    n = (z_alpha + z_beta)**2 * p_pooled * (1 - p_pooled) / (effect_size**2)
    return math.ceil(n)

# example
required_n = sample_size_calculator(0.082812, 0.029412)
print(f"Required sample size per group: {required_n:,}")

Required sample size per group: 146


DAYS NEEDED TO ACHIEVE RESULTS

In [5]:
# Let's say your true baseline is around 2% and you want to detect a 50% relative change
baseline_rate = 0.02  # 2% baseline
min_detectable_effect = 0.01  # 1% absolute change (50% relative)

required_n = sample_size_calculator(baseline_rate, baseline_rate + min_detectable_effect)
print(f"For 2% → 3% detection: {required_n:,} users per group")

# Your current traffic rate
current_daily_users = (640 + 1530) / 16  # assuming 17 days based on dates
print(f"Current daily traffic: ~{current_daily_users:.0f} users")
print(f"Days needed: {(required_n * 2) / current_daily_users:.0f}")

For 2% → 3% detection: 1,914 users per group
Current daily traffic: ~136 users
Days needed: 28


In [6]:
from statsmodels.stats.proportion import proportions_ztest

STATISTICS ON CONTROL AND TEST GROUPS

In [7]:
from statsmodels.stats.proportion import proportions_ztest

control_conversions = 53    # people who converted in control group
control_total = 685       # total people in control group

test_conversions = 46     # people who converted in test group  
test_total = 1622         # total people in test

# Two-proportion z-test
successes = [control_conversions, test_conversions]  # [17, 7]
totals = [control_total, test_total] # [685, 1622]

z_stat, p_value = proportions_ztest(successes, totals)

print(f"Control group conversion rate: {control_conversions/control_total:.3f}")
print(f"test group conversion rate: {test_conversions/test_total:.3f}")
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.12f}")

Control group conversion rate: 0.077
test group conversion rate: 0.028
Z-statistic: 5.3074
P-value: 0.000000111201


AB TEST STATISTICS ANALYSIS

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import beta
import math
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from statsmodels.stats.power import ttest_power

# Your current data
control_users = 640
control_conversions = 53  # unique users with shipments
test_users = 1530
test_conversions = 45

# Also shipment rates for additional analysis
control_shipments = 53
test_shipments = 45

print("=" * 60)
print("COMPREHENSIVE A/B TEST ANALYSIS")
print("=" * 60)

# 1. BASIC STATISTICS
print("\n1. BASIC STATISTICS")
print("-" * 30)
control_rate = control_conversions / control_users
test_rate = test_conversions / test_users
difference = test_rate - control_rate
relative_change = (difference / control_rate) * 100

print(f"Control: {control_conversions}/{control_users} = {control_rate:.4f} ({control_rate*100:.2f}%)")
print(f"Test: {test_conversions}/{test_users} = {test_rate:.4f} ({test_rate*100:.2f}%)")
print(f"Absolute difference: {difference:.4f} ({difference*100:.2f} pp)")
print(f"Relative change: {relative_change:.1f}%")

# 2. STATISTICAL SIGNIFICANCE TESTING
print("\n2. STATISTICAL SIGNIFICANCE")
print("-" * 30)

# Two-proportion z-test
successes = [control_conversions, test_conversions]
totals = [control_users, test_users]
z_stat, p_value = proportions_ztest(successes, totals)

print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.12f}")
print(f"Significant at α=0.05: {'YES' if p_value < 0.05 else 'NO'}")

# 3. CONFIDENCE INTERVALS
print("\n3. CONFIDENCE INTERVALS")
print("-" * 30)

# Individual CIs
control_ci = proportion_confint(control_conversions, control_users, alpha=0.05, method='wilson')
test_ci = proportion_confint(test_conversions, test_users, alpha=0.05, method='wilson')

print(f"Control 95% CI: [{control_ci[0]:.4f}, {control_ci[1]:.4f}] = [{control_ci[0]*100:.2f}%, {control_ci[1]*100:.2f}%]")
print(f"Test 95% CI: [{test_ci[0]:.4f}, {test_ci[1]:.4f}] = [{test_ci[0]*100:.2f}%, {test_ci[1]*100:.2f}%]")

# Difference CI (approximation)
se_diff = np.sqrt(control_rate*(1-control_rate)/control_users + test_rate*(1-test_rate)/test_users)
diff_ci_lower = difference - 1.96 * se_diff
diff_ci_upper = difference + 1.96 * se_diff
print(f"Difference 95% CI: [{diff_ci_lower:.4f}, {diff_ci_upper:.4f}] = [{diff_ci_lower*100:.2f}pp, {diff_ci_upper*100:.2f}pp]")

# 4. EFFECT SIZE (Cohen's h for proportions)
print("\n4. EFFECT SIZE")
print("-" * 30)
cohens_h = 2 * (np.arcsin(np.sqrt(control_rate)) - np.arcsin(np.sqrt(test_rate)))
print(f"Cohen's h: {cohens_h:.4f}")
if abs(cohens_h) < 0.2:
    effect_magnitude = "Small"
elif abs(cohens_h) < 0.5:
    effect_magnitude = "Medium"
else:
    effect_magnitude = "Large"
print(f"Effect magnitude: {effect_magnitude}")

# 5. POWER ANALYSIS FOR CURRENT TEST
print("\n5. CURRENT POWER ANALYSIS")
print("-" * 30)

def calculate_power(n1, n2, p1, p2, alpha=0.05):
    """Calculate statistical power for two-proportion test"""
    pooled_p = (n1*p1 + n2*p2) / (n1 + n2)
    se_null = np.sqrt(pooled_p * (1 - pooled_p) * (1/n1 + 1/n2))
    se_alt = np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
    
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = (abs(p1 - p2) - z_alpha * se_null) / se_alt
    power = stats.norm.cdf(z_beta)
    
    return max(0, min(1, power))

current_power = calculate_power(control_users, test_users, control_rate, test_rate)
print(f"Current statistical power: {current_power:.3f} ({current_power*100:.1f}%)")

# 6. BAYESIAN ANALYSIS
print("\n6. BAYESIAN ANALYSIS")
print("-" * 30)

# Beta-binomial model with uniform priors
alpha_control = 1 + control_conversions
beta_control = 1 + control_users - control_conversions
alpha_test = 1 + test_conversions
beta_test = 1 + test_users - test_conversions

# Probability that test is worse than control
n_samples = 100000
control_samples = np.random.beta(alpha_control, beta_control, n_samples)
test_samples = np.random.beta(alpha_test, beta_test, n_samples)

prob_test_worse = np.mean(test_samples < control_samples)
prob_test_much_worse = np.mean(test_samples < 0.5 * control_samples)  # >50% worse

print(f"Probability test is worse than control: {prob_test_worse:.4f} ({prob_test_worse*100:.1f}%)")
print(f"Probability test is >50% worse: {prob_test_much_worse:.4f} ({prob_test_much_worse*100:.1f}%)")

# 7. SAMPLE SIZE FOR FUTURE TESTS
print("\n7. SAMPLE SIZE PLANNING FOR FUTURE TESTS")
print("-" * 30)

def sample_size_calculator(p1, p2, alpha=0.05, power=0.8):
    """Calculate required sample size per group"""
    p_pooled = (p1 + p2) / 2
    effect_size = abs(p1 - p2)
    
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    
    n = (z_alpha + z_beta)**2 * p_pooled * (1 - p_pooled) / (effect_size**2)
    return math.ceil(n)

# Scenarios for future tests (assuming control rate is true baseline)
baseline = control_rate
scenarios = [
    ("Detect 10% relative change", baseline * 1.1),
    ("Detect 20% relative change", baseline * 1.2),
    ("Detect 50% relative change", baseline * 1.5),
    ("Detect current difference", test_rate)
]

for scenario_name, p2 in scenarios:
    if p2 > 0:
        n_needed = sample_size_calculator(baseline, p2)
        days_needed = (n_needed * 2) / ((control_users + test_users) / 17)  # 17 day test
        print(f"{scenario_name}: {n_needed:,} users per group ({days_needed:.0f} days)")

# 8. EARLY STOPPING RECOMMENDATION
print("\n8. EARLY STOPPING ANALYSIS")
print("-" * 30)

# Check if we have sufficient evidence to stop
min_sample_for_significance = 30  # Rule of thumb minimum
total_conversions = control_conversions + test_conversions

print(f"Total conversions: {total_conversions}")
print(f"Minimum for reliable test: {min_sample_for_significance}")
print(f"Current power: {current_power*100:.1f}%")
print(f"P-value: {p_value:.4f}")

if total_conversions >= min_sample_for_significance and p_value < 0.05:
    print("✅ STOP RECOMMENDATION: Sufficient evidence of significant negative effect")
elif prob_test_worse > 0.95:
    print("✅ STOP RECOMMENDATION: Bayesian analysis shows >95% probability test is worse")
else:
    print("⚠️ CONTINUE RECOMMENDATION: Insufficient evidence (but this shouldn't apply to your case)")

# 9. BUSINESS IMPACT ANALYSIS
print("\n9. BUSINESS IMPACT")
print("-" * 30)

# Lost conversions due to skewed allocation
if test_users > control_users:  # More traffic went to worse variant
    expected_conversions_if_equal = control_rate * test_users
    actual_test_conversions = test_conversions
    lost_conversions = expected_conversions_if_equal - actual_test_conversions
    
    print(f"Expected conversions if test users got control rate: {expected_conversions_if_equal:.1f}")
    print(f"Actual test conversions: {actual_test_conversions}")
    print(f"Lost conversions due to poor allocation: {lost_conversions:.1f}")
    print(f"Opportunity cost: {(lost_conversions/(control_conversions + test_conversions))*100:.1f}% of total conversions")

# 10. FINAL RECOMMENDATIONS
print("\n10. FINAL RECOMMENDATIONS")
print("-" * 30)
print("Based on the analysis:")
print()

if prob_test_worse > 0.95 and p_value < 0.05:
    print("🚨 IMMEDIATE ACTION REQUIRED:")
    print("   • STOP the test immediately")
    print("   • Route 100% traffic to control")
    print("   • Test variant is definitively worse")
    print()
    print("🔍 ROOT CAUSE INVESTIGATION:")
    print("   • Test variant has fundamental issues")
    print("   • Investigate what changed in promoted button")
    print("   • Fix core functionality before testing variants")
    print()
    print("📊 FUTURE TESTING:")
    print("   • Fix traffic allocation (50/50 split)")
    print("   • Plan for larger samples for smaller effects")
    print("   • Consider staged rollouts for major changes")
else:
    print("📈 Continue testing with more data")

print("\n" + "=" * 60)

: 

## POWER BI DATA EXPORT

Export the analysis results to CSV files for Power BI dashboard creation:

In [ ]:
# Create Power BI Export Data
import os
import pandas as pd

# Create output directory
output_dir = 'powerbi_data'
os.makedirs(output_dir, exist_ok=True)

# 1. Summary Statistics for Power BI
summary_data = {
    'Metric': [
        'Control Users',
        'Test Users', 
        'Control Conversions',
        'Test Conversions',
        'Control Rate',
        'Test Rate',
        'Absolute Difference',
        'Relative Change %',
        'Z Statistic',
        'P Value',
        'Statistical Significant',
        'Effect Size (Cohen h)',
        'Current Power',
        'Prob Test Worse than Control'
    ],
    'Value': [
        control_users,
        test_users,
        control_conversions, 
        test_conversions,
        f"{control_rate:.4f}",
        f"{test_rate:.4f}",
        f"{difference:.4f}",
        f"{relative_change:.2f}",
        f"{z_stat:.4f}",
        f"{p_value:.6f}",
        'Yes' if p_value < 0.05 else 'No',
        f"{cohens_h:.4f}",
        f"{current_power:.3f}",
        f"{prob_test_worse:.3f}"
    ],
    'Category': [
        'Sample Size', 'Sample Size', 'Conversions', 'Conversions',
        'Rate', 'Rate', 'Difference', 'Performance',
        'Statistical Test', 'Statistical Test', 'Result', 'Effect',
        'Power', 'Bayesian'
    ]
}

summary_df = pd.DataFrame(summary_data)

# 2. Confidence Intervals Data
ci_data = pd.DataFrame({
    'Group': ['Control', 'Test'],
    'Conversion_Rate': [control_rate, test_rate],
    'Users': [control_users, test_users],
    'Conversions': [control_conversions, test_conversions],
    'CI_Lower': [control_ci[0], test_ci[0]],
    'CI_Upper': [control_ci[1], test_ci[1]],
    'CI_Width': [control_ci[1] - control_ci[0], test_ci[1] - test_ci[0]]
})

# 3. Sample Size Planning Data
sample_size_scenarios = []
baseline = control_rate
scenarios = [
    ("10% Relative Improvement", baseline * 1.1),
    ("20% Relative Improvement", baseline * 1.2), 
    ("50% Relative Improvement", baseline * 1.5),
    ("Current Observed Difference", test_rate)
]

for scenario, target_rate in scenarios:
    if target_rate > 0:
        n_needed = sample_size_calculator(baseline, target_rate)
        sample_size_scenarios.append({
            'Scenario': scenario,
            'Target_Rate': target_rate,
            'Required_Sample_Size': n_needed,
            'Days_Needed': (n_needed * 2) / ((control_users + test_users) / 17)
        })

sample_size_df = pd.DataFrame(sample_size_scenarios)

# 4. Business Impact Data
business_impact = pd.DataFrame({
    'Metric': [
        'Expected Conversions (if equal split)',
        'Actual Test Conversions',
        'Lost Conversions',
        'Opportunity Cost %'
    ],
    'Value': [
        expected_conversions_if_equal,
        actual_test_conversions,
        lost_conversions,
        (lost_conversions/(control_conversions + test_conversions))*100
    ]
}) if test_users > control_users else pd.DataFrame({
    'Metric': ['No business impact calculated - control had more users'],
    'Value': [0]
})

# Export all files
summary_df.to_csv(f'{output_dir}/ab_test_summary.csv', index=False)
ci_data.to_csv(f'{output_dir}/confidence_intervals.csv', index=False)
sample_size_df.to_csv(f'{output_dir}/sample_size_planning.csv', index=False)
business_impact.to_csv(f'{output_dir}/business_impact.csv', index=False)

print("✅ Power BI data files created successfully!")
print(f"📂 Files saved in: {output_dir}/")
print("   • ab_test_summary.csv - Key metrics and results")
print("   • confidence_intervals.csv - Statistical confidence data")
print("   • sample_size_planning.csv - Future test planning")
print("   • business_impact.csv - Business impact analysis")

print("\n📊 Summary Data Preview:")
print(summary_df.head(10))

print("\n📈 Confidence Intervals Preview:")
print(ci_data)